<a href="https://colab.research.google.com/github/sinansonmez/Coursera_Capstone/blob/master/immobiliare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from urllib.request import urlopen
import pandas as pd
import numpy as np
!pip install bs4
from bs4 import BeautifulSoup

In [3]:
# Use the requests library to download the webpage
base_url = "https://www.immobiliare.it/vendita-case/"
municipality = "lainate"
page = "/?pag="
url = base_url + municipality
my_headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.1 Safari/605.1.15',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'
}

html_data = requests.get(url, headers=my_headers).text

In [4]:
# Parse the html data using beautiful_soup
soup = BeautifulSoup(html_data, "html5lib")
page_num = soup.find_all('li', class_='disabled')[-1].text

In [5]:
# Using beautiful soup extract the table and store it into a dataframe named istanbul_data. 
# The dataframe have columns. Fill in each variable with the correct data from the list cell.

real_estate_data = pd.DataFrame(columns=['id', 'title', 'price', 'room', 'sqm', 'bagni', 'piano', 'link'])

# for each page get the data and append to real_estate_data
for num in range(1, int(page_num) + 1):
  url_page_num = base_url + municipality + page + str(num)
  soup = BeautifulSoup(urlopen(url_page_num), "html5lib")

  list = soup.find_all('div', class_='listing-item_body--content')

  for item in list:
    title = item.find('p').text.replace('\n','')
    link = item.find('a').get('href').replace('\n','')
    id = link.split('/')[4]
    for detail in item.find_all('li'):
      if (detail.text.find('€') != -1):
        price = detail.text.replace('€ ','').replace('.','').replace(' ','').replace('da','').replace('\n','')
      if (detail.text.find('locali') != -1):
        room = detail.text.replace(' ', '').replace('locali', '').replace('\n','')
      if (detail.text.find('superfici') != -1):
        sqm = detail.text.replace('m2superficie', '').replace('\n','')
      if (detail.text.find('bagni') != -1):
        bagni = detail.text.replace('bagni', '').replace('\n','')
      if (detail.text.find('piano') != -1):
        piano = detail.text.replace('piano', '').replace('\n','')
    
    real_estate_data = real_estate_data.append({'id':id, 'title': title, 'price': price, 'room': room, 'sqm': sqm, 'bagni': bagni, 'piano': piano, 'link': link}, ignore_index=True)

In [12]:
# real_estate_data
real_estate_data.to_csv('real_estate_data.csv', index=False)